In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import AUC, CategoricalAccuracy
from tensorflow.keras.optimizers import Adam

import pandas as pd
import matplotlib.pyplot as plt

In [24]:
train_set = pd.read_csv('../input/human-action-recognition-har-dataset/Human Action Recognition/Training_set.csv')
test_set = pd.read_csv('../input/human-action-recognition-har-dataset/Human Action Recognition/Testing_set.csv')

In [25]:
BATCH_SIZE = 16
EPOCHS=10

In [26]:
from keras.preprocessing.image import ImageDataGenerator

# Loading Training Data 
train_image_generator = ImageDataGenerator(rescale=1./255, horizontal_flip=True, shear_range=0.2, zoom_range=0.2, width_shift_range=0.05, height_shift_range=0.05)
train_iterator = train_image_generator.flow_from_dataframe(dataframe= train_set, directory='/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/train/', x_col='filename', y_col='label', class_mode='categorical')


Found 12600 validated image filenames belonging to 15 classes.


In [27]:
# Loading Validation Data.....

validation_image_generator = ImageDataGenerator(rescale=1./255)
validation_iterator = validation_image_generator.flow_from_dataframe(dataframe=test_set, directory='../input/human-action-recognition-har-dataset/Human Action Recognition/test/',x_col='filename', y_col=None, class_mode=None)
# idg_test.flow_from_dataframe(dataframe=test_class_data,directory="/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/test/",x_col='filename',y_col=None,target_size=(64,64),batch_size=32,class_mode=None) 

Found 5400 validated image filenames.


In [28]:
train_set.label.nunique()

15

In [29]:
# Defining Model

model = Sequential([
    Input(shape=(256,256,3)),
    
    Conv2D(8, 5, strides=3, activation='relu'),
    MaxPooling2D(pool_size=(5,5), strides=(5,5), padding='valid'),
    
    Conv2D(16, 5, strides =2, activation='relu'),
    MaxPooling2D(pool_size=(3,3), strides=(3,3), padding='valid'),
    
    Flatten(),
    
    Dense(128, activation='relu'),
    
    
    Dense(15,activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 84, 84, 8)         608       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 6, 6, 16)          3216      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 2, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_7 (Dense)              (None, 15)               

In [30]:
# Compiling model.... 


model.compile(
        optimizer=Adam(learning_rate=0.1),
        loss=CategoricalCrossentropy(),
        metrics=['accuracy'])

# metrics=[AUC(), CategoricalAccuracy()])

In [ ]:
# Fitting Model 

# history = model.fit(train_iterator, 
#                    steps_per_epoch = train_iterator.samples/BATCH_SIZE,
#                    epochs=EPOCHS,
#                    validation_data=validation_iterator,
#                    validation_steps=validation_iterator.samples/BATCH_SIZE)


history = model.fit(x=train_iterator, validation_data=validation_iterator, epochs=5)

Epoch 1/5
394/394 [==============================] - 225s 571ms/step - loss: 3.1033 - accuracy: 0.0640 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
394/394 [==============================] - 249s 633ms/step - loss: 2.7240 - accuracy: 0.0671 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/5
366/394 [==========================>...] - ETA: 14s - loss: 2.7224 - accuracy: 0.0666

In [ ]:
# Predictions 

from tensorflow.keras.preprocessing import image

root_path = '/kaggle/input/human-action-recognition-har-dataset/Human Action Recognition/test/'